In [2]:
import numpy as np

In [4]:
src = r"D:\3d-reconstruction\datasets\splatam-nerf-03052024\params.npz"

In [5]:
params = np.load(src)

xyz     = params['means3D']
normals = np.zeros_like(xyz)
f_dc    = params['rgb_colors']
opacities = params['logit_opacities']
scale     = params['log_scales'].repeat(3, axis=-1)
rotation  = params['unnorm_rotations']

In [6]:
np.save("orig_xyz_from_npz.npy", xyz)
np.save("orig_rgb_from_npz.npy", f_dc)

In [ ]:
## Edit points in params.npz to match the points that we got from training 3DGS

In [79]:
print(list(params.keys()))

['means3D', 'rgb_colors', 'unnorm_rotations', 'logit_opacities', 'log_scales', 'cam_unnorm_rots', 'cam_trans', 'timestep', 'intrinsics', 'w2c', 'org_width', 'org_height', 'gt_w2c_all_frames', 'keyframe_time_indices']


In [80]:
for k in list(params.keys()) :
    print(k, params[k].shape)

means3D (861743, 3)
rgb_colors (861743, 3)
unnorm_rotations (861743, 4)
logit_opacities (861743, 1)
log_scales (861743, 1)
cam_unnorm_rots (1, 4, 50)
cam_trans (1, 3, 50)
timestep (861743,)
intrinsics (3, 3)
w2c (4, 4)
org_width ()
org_height ()
gt_w2c_all_frames (50, 4, 4)
keyframe_time_indices (12,)


In [81]:
params['gt_w2c_all_frames'][0,...]

array([[ 1.0000000e+00,  1.8626451e-09,  2.7755579e-17, -4.4703484e-08],
       [-1.8626454e-09,  1.0000000e+00,  1.4901163e-08,  1.4901161e-08],
       [-1.4901163e-08, -2.7755579e-17,  1.0000001e+00, -1.4901161e-08],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32)

In [8]:
import torch
from scipy.spatial.transform import Rotation


first_frame_R = torch.tensor(params['cam_unnorm_rots'][:,:,0])
print(first_frame_R)

r = Rotation.from_rotvec(np.pi/2 * np.array([0, 0, 1]))
R_matrix = r.as_matrix()
print(R_matrix)

first_frame_T = torch.tensor(params['cam_trans'][:,:,0])
print(first_frame_T)

first_frame_w2c = torch.tensor(params['w2c'])
print(first_frame_w2c)
print(first_frame_w2c.shape) 

c:\Users\chana\anaconda3\envs\gs_viser\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[1., 0., 0., 0.]])
[[ 2.22044605e-16 -1.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  2.22044605e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
tensor([[0., 0., 0.]])
tensor([[ 1.0000e+00,  1.8626e-09,  2.7756e-17, -4.4703e-08],
        [-1.8626e-09,  1.0000e+00,  1.4901e-08,  1.4901e-08],
        [-1.4901e-08, -2.7756e-17,  1.0000e+00, -1.4901e-08],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])
torch.Size([4, 4])


In [17]:
import torch
first_frame_c2w =  torch.linalg.inv((torch.tensor(first_frame_w2c.clone().detach())))
print(first_frame_c2w)

tensor([[ 1.0000e+00, -1.8626e-09, -0.0000e+00,  4.4703e-08],
        [ 1.8626e-09,  1.0000e+00, -1.4901e-08, -1.4901e-08],
        [ 1.4901e-08,  0.0000e+00,  1.0000e+00,  1.4901e-08],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])


C:\Users\chana\AppData\Local\Temp\ipykernel_17704\2887472891.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  first_frame_c2w =  torch.linalg.inv((torch.tensor(first_frame_w2c.clone().detach())))


In [22]:
points = torch.tensor(params['means3D'])
ones = torch.ones((points.shape[0], 1))

points_ones = torch.cat((points, ones), dim=1)

print(points_ones)
print(points_ones.shape)

tensor([[-0.9940, -0.7202,  1.3847,  1.0000],
        [-0.9648, -0.7453,  1.4508,  1.0000],
        [-0.9591, -0.7529,  1.4637,  1.0000],
        ...,
        [-2.0291, -0.5660,  3.0558,  1.0000],
        [-1.6353, -0.4126,  2.9322,  1.0000],
        [-1.6313, -0.3694,  2.8564,  1.0000]])
torch.Size([861743, 4])


In [23]:
# Invert the W2C transformation matrix to get C2W
C2W_matrix = np.linalg.inv(first_frame_w2c)

# Convert points from camera space to world space
points_world = np.dot(C2W_matrix, points_ones.T).T[:, :3]
print(points_world)


np.save("points_world.npy", points_world)

[[-0.9940368  -0.72022104  1.384701  ]
 [-0.9647725  -0.7452825   1.450773  ]
 [-0.95910203 -0.75287193  1.4637486 ]
 ...
 [-2.029058   -0.5659783   3.055822  ]
 [-1.6353457  -0.4126091   2.9322371 ]
 [-1.6313092  -0.369389    2.8564053 ]]


In [27]:
torch.eq(points, torch.tensor(points_world))

tensor([[False,  True, False],
        [False,  True, False],
        [False,  True, False],
        ...,
        [ True, False, False],
        [ True, False, False],
        [ True, False, False]])

In [13]:
C2W_matrix

array([[ 1.0000000e+00, -1.8626451e-09,  0.0000000e+00,  4.4703484e-08],
       [ 1.8626451e-09,  1.0000000e+00, -1.4901161e-08, -1.4901161e-08],
       [ 1.4901161e-08,  0.0000000e+00,  9.9999988e-01,  1.4901160e-08],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32)

In [28]:
import numpy as np

def transform_point(point, transformation_matrix):
    # Convert the point to homogeneous coordinates (append 1)
    point = np.append(point, 1)
    
    # Perform matrix multiplication
    transformed_point = np.dot(transformation_matrix, point)
    
    # Normalize the result by dividing by the fourth component
    transformed_point /= transformed_point[-1]
    
    # Return the transformed point without the homogeneous coordinate
    return transformed_point[:-1]


# Example 3D point
point = np.array([1,2,3])

# Apply transformation
transformed_point = transform_point(point, transformation_matrix)

print("Original point:", point)
print("Transformed point:", transformed_point)

Original point: [-0.39235845 -0.2755367   1.3324081 ]
Transformed point: [-0.39235841 -0.27553672  1.33240792]


In [15]:
points_c2w = torch.matmul(points_ones, first_frame_c2w)
print(points_c2w)

tensor([[-0.9940, -0.7202,  1.3847,  1.0000],
        [-0.9648, -0.7453,  1.4508,  1.0000],
        [-0.9591, -0.7529,  1.4637,  1.0000],
        ...,
        [-2.0291, -0.5660,  3.0558,  1.0000],
        [-1.6353, -0.4126,  2.9322,  1.0000],
        [-1.6313, -0.3694,  2.8564,  1.0000]])


In [86]:
points_w2c = torch.matmul(points_ones, first_frame_w2c)
print(points_w2c)

tensor([[-0.9940, -0.7202,  1.3847,  1.0000],
        [-0.9648, -0.7453,  1.4508,  1.0000],
        [-0.9591, -0.7529,  1.4637,  1.0000],
        ...,
        [-2.0291, -0.5660,  3.0558,  1.0000],
        [-1.6353, -0.4126,  2.9322,  1.0000],
        [-1.6313, -0.3694,  2.8564,  1.0000]])


In [62]:
# transformed_point_cloud = (first_frame_R @ points) + first_frame_T

In [56]:
first_frame_c2w

tensor([[ 1.0000e+00, -1.8626e-09, -0.0000e+00,  4.4703e-08],
        [ 1.8626e-09,  1.0000e+00, -1.4901e-08, -1.4901e-08],
        [ 1.4901e-08,  0.0000e+00,  1.0000e+00,  1.4901e-08],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])

In [57]:
first_frame_w2c

tensor([[ 1.0000e+00,  1.8626e-09,  2.7756e-17, -4.4703e-08],
        [-1.8626e-09,  1.0000e+00,  1.4901e-08,  1.4901e-08],
        [-1.4901e-08, -2.7756e-17,  1.0000e+00, -1.4901e-08],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])

In [40]:
print(torch.matmul(transform_matrix_first_frame,first_frame_c2w))
print(torch.matmul(transform_matrix_first_frame,first_frame_w2c))

tensor([[-0.0026,  0.9902, -0.1395, -0.1498],
        [-0.9880,  0.0190,  0.1535,  0.2498],
        [ 0.1546,  0.1382,  0.9783,  0.0756],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
tensor([[-0.0026,  0.9902, -0.1395, -0.1498],
        [-0.9880,  0.0190,  0.1535,  0.2498],
        [ 0.1546,  0.1382,  0.9783,  0.0756],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])


In [39]:
print(torch.matmul(first_frame_c2w @ transform_matrix_first_frame))
print(torch.matmul(first_frame_w2c @ transform_matrix_first_frame))

tensor([[-0.0026,  0.9902, -0.1395, -0.1498],
        [-0.9880,  0.0190,  0.1535,  0.2498],
        [ 0.1546,  0.1382,  0.9783,  0.0756],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
tensor([[-0.0026,  0.9902, -0.1395, -0.1498],
        [-0.9880,  0.0190,  0.1535,  0.2498],
        [ 0.1546,  0.1382,  0.9783,  0.0756],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])


In [ ]:
# W2C
def getWorld2View2(R, t, translate=np.array([.0, .0, .0]), scale=1.0):
    Rt = np.zeros((4, 4))
    Rt[:3, :3] = R.transpose()
    Rt[:3, 3] = t
    Rt[3, 3] = 1.0

    C2W = np.linalg.inv(Rt)
    cam_center = C2W[:3, 3]
    cam_center = (cam_center + translate) * scale
    C2W[:3, 3] = cam_center
    Rt = np.linalg.inv(C2W)
    return np.float32(Rt)

In [ ]:
import numpy as np
 
# Define the camera transformation matrix
M_camera = np.array([
    [r11, r12, r13, tx],
    [r21, r22, r23, ty],
    [r31, r32, r33, tz],
    [0,   0,   0,   1 ]
])
 
# Define the point in camera space (homogeneous coordinates)
P_camera = np.array([x, y, z, 1])
 
# Transform the point to world space
P_world = np.dot(M_camera, P_camera)
 
# Convert back to Cartesian coordinates if needed
P_world_cartesian = P_world[:3] / P_world[3]
 
print("Point in world space:", P_world_cartesian)

In [368]:
# cameras.txt

# height width
import json
json_file = open('../transforms.json')
data = json.load(json_file)
height = data['h']
width  = data['w']

FX = data['fl_x']
FY  = data['fl_y']
CX = data['cx']
CY = data['cy']
print(CX, CY, FX, FY)

f = open("cameras.txt", "w")
f.write(f"1 PINHOLE {width} {height} {FX} {FY} {CX} {CY}")
f.close()

966.8006591796875 722.9915161132812 1338.148681640625 1338.148681640625


In [369]:
from os import listdir
from os.path import isfile, join
path = r"D:\3d-reconstruction\SplaTAM\experiments\iPhone_Captures\splatam_demo-2\rgb"
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
onlyfiles.sort()
print(onlyfiles)

['00.png', '01.png', '02.png', '03.png', '04.png', '05.png', '06.png', '07.png', '08.png', '09.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png', '31.png', '32.png', '33.png', '34.png', '35.png', '36.png', '37.png', '38.png', '39.png', '40.png', '41.png', '42.png', '43.png', '44.png', '45.png', '46.png', '47.png', '48.png', '49.png', '50.png', '51.png', '52.png', '53.png', '54.png', '55.png', '56.png', '57.png', '58.png', '59.png']


In [370]:
# # images.txt
# # IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME

# f = open("images.txt", "w")

# for idx in range(params['cam_unnorm_rots'].shape[2]):
#     IMAGE_ID = idx+1

#     QW, QX, QY, QZ  = params['cam_unnorm_rots'][:,:,idx].squeeze(0)
#     TX, TY, TZ = params['cam_trans'][:,:,idx].squeeze(0) * 10
#     CAMERA_ID = 1
#     NAME = onlyfiles[idx]

#     mylist = ' '.join([str(i) for i in [IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME]])

#     f.write(mylist+'\n')

#     # dummy
#     f.write('0 0 0 0 0 0'+'\n')

# f.close()

In [371]:
def rotation_matrix_to_quaternion(rotation_matrix):
    """
    Convert a 3x3 rotation matrix to a quaternion.

    :param rotation_matrix: A 3x3 numpy array representing the rotation matrix.
    :return: A list representing the quaternion in the order [w, x, y, z].
    """
    trace = rotation_matrix[0, 0] + rotation_matrix[1, 1] + rotation_matrix[2, 2]

    if trace > 0:
        S = np.sqrt(trace + 1.0) * 2
        qw = 0.25 * S
        qx = (rotation_matrix[2, 1] - rotation_matrix[1, 2]) / S
        qy = (rotation_matrix[0, 2] - rotation_matrix[2, 0]) / S
        qz = (rotation_matrix[1, 0] - rotation_matrix[0, 1]) / S
    elif (rotation_matrix[0, 0] > rotation_matrix[1, 1]) and (rotation_matrix[0, 0] > rotation_matrix[2, 2]):
        S = np.sqrt(1.0 + rotation_matrix[0, 0] - rotation_matrix[1, 1] - rotation_matrix[2, 2]) * 2
        qw = (rotation_matrix[2, 1] - rotation_matrix[1, 2]) / S
        qx = 0.25 * S
        qy = (rotation_matrix[0, 1] + rotation_matrix[1, 0]) / S
        qz = (rotation_matrix[0, 2] + rotation_matrix[2, 0]) / S
    elif rotation_matrix[1, 1] > rotation_matrix[2, 2]:
        S = np.sqrt(1.0 + rotation_matrix[1, 1] - rotation_matrix[0, 0] - rotation_matrix[2, 2]) * 2
        qw = (rotation_matrix[0, 2] - rotation_matrix[2, 0]) / S
        qx = (rotation_matrix[0, 1] + rotation_matrix[1, 0]) / S
        qy = 0.25 * S
        qz = (rotation_matrix[1, 2] + rotation_matrix[2, 1]) / S
    else:
        S = np.sqrt(1.0 + rotation_matrix[2, 2] - rotation_matrix[0, 0] - rotation_matrix[1, 1]) * 2
        qw = (rotation_matrix[1, 0] - rotation_matrix[0, 1]) / S
        qx = (rotation_matrix[0, 2] + rotation_matrix[2, 0]) / S
        qy = (rotation_matrix[1, 2] + rotation_matrix[2, 1]) / S
        qz = 0.25 * S

    return [qw, qx, qy, qz]

In [372]:
# images.txt
# IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME

f = open("images.txt", "w")

for idx in range(params['gt_w2c_all_frames'].shape[0]):
    IMAGE_ID = idx+1

    w2c = params['gt_w2c_all_frames'][idx]

    R = w2c[:3,:3]
    T = w2c[:3, 3]

    Qs = rotation_matrix_to_quaternion(R)

    CAMERA_ID = 1
    NAME = onlyfiles[idx]

    mylist = ' '.join([str(i) for i in [IMAGE_ID, Qs[0], Qs[1], Qs[2], Qs[3], T[0], T[1], T[2], CAMERA_ID, NAME]])
    print(mylist)

    f.write(mylist+'\n')

    # dummy
    f.write('0 0 0 0 0 0'+'\n')

f.close()

1 1.0 1.862645149230957e-09 3.725290742551124e-09 -9.313225746154785e-10 0.0 0.0 0.0 1 00.png
2 0.9918102538448599 0.1248251643725743 0.021467456399628267 0.016440084884080285 -0.0030356357 -0.07371941 0.013058394 1 01.png
3 0.991192837253919 0.1307947450033889 0.01970630983245455 0.006413024411011927 -8.426808e-05 -0.52367675 0.0147223985 1 02.png
4 0.9730948259873299 0.2280475360338746 0.018920536491780663 0.02688495165754793 0.024811575 -0.68957746 -0.1317807 1 03.png
5 0.9277374588764534 0.367831515266256 0.009008783209280837 0.0626261265673519 0.071061075 -0.63449955 -0.33270422 1 04.png
6 0.8182465468676764 0.5637574232788365 0.006159484484291634 0.11230452988329609 0.12537974 -0.43044654 -0.56622684 1 05.png
7 0.8978267086191352 0.430666862589467 -0.0019113577241891724 0.09181254597200428 0.25508532 -1.2046912 -1.0969373 1 06.png
8 0.9476684972061294 0.3127856608009215 -0.011619602855827655 0.06288470758538245 0.17149061 -1.1836982 -0.78066117 1 07.png
9 0.9820379944344401 0.186

In [373]:
# def quaternion_to_rotation_matrix(quaternion):
#     """     Convert a quaternion to a 3x3 rotation matrix.
#     :param quaternion: A list or numpy array representing the quaternion in the order [w, x, y, z].
#     :return: A 3x3 numpy array representing the rotation matrix.     """
#     w, x, y, z = quaternion
#     rotation_matrix = np.array([ 
#         [1 - 2*y**2 - 2*z**2, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
#         [2*x*y + 2*z*w, 1 - 2*x**2 - 2*z**2, 2*y*z - 2*x*w],
#         [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x**2 - 2*y**2]     ])
#     return rotation_matrix

In [374]:
# # images.txt
# # IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME

# f = open("images.txt", "w")

# for idx in range(params['cam_unnorm_rots'].shape[2]):
#     IMAGE_ID = idx+1

#     QW, QX, QY, QZ  = params['cam_unnorm_rots'][:,:,idx].squeeze(0)
#     TX, TY, TZ = params['cam_trans'][:,:,idx].squeeze(0)

#     rotation_matrix = quaternion_to_rotation_matrix((QW, QX, QY, QZ))
#     # print(rotation_matrix.shape)

#     Q = np.transpose(rotation_matrix) # (3,3)
#     # print(Q.shape)

#     T = -Q @ np.array([TX, TY, TZ]).reshape(3,1) # (3,3) @ (3,1)
#     # print(T.shape)

#     QW, QX, QY, QZ = rotation_matrix_to_quaternion(Q)

#     CAMERA_ID = 1
#     NAME = onlyfiles[idx]

#     mylist = ' '.join([str(i) for i in [IMAGE_ID, QW, QX, QY, QZ, T[0][0], T[1][0], T[2][0], CAMERA_ID, NAME]])

#     f.write(mylist+'\n')

#     # dummy
#     f.write('0 0 0 0 0 0'+'\n')

# f.close()

In [375]:
# Point3D params
xyz     = params['means3D']
colors    = params['rgb_colors']
rotation  = params['unnorm_rotations']
scale     = params['log_scales'].repeat(3, axis=-1)
opacity   = params['logit_opacities']

In [376]:
f = open("points3D.txt", "w")

for idx in range(xyz.shape[0]):
    POINT3D_ID = idx+1
    X, Y, Z = xyz[idx, :]

    R, G, B = colors[idx, :]*255
    R, G, B = int(R), int(G), int(B) 


    # dummy
    error = 0
    point2d_idx = 0

    mylist = ' '.join([str(i) for i in [POINT3D_ID, X, Y, Z, R, G, B, error, point2d_idx, point2d_idx]])

    f.write(mylist+'\n')

f.close()

In [377]:
# # Camera rotation each image
# cam_unnorm_rots = params['cam_unnorm_rots']
# for i in range(10):
#     print(cam_unnorm_rots[:,:,i])

# # # Camera transform each image
# # cam_trans = params['cam_trans']

# # for i in range(10):
# #     print(cam_trans[:,:,i])

In [378]:
params['gt_w2c_all_frames'][4]

array([[ 0.99199367, -0.10957377,  0.06278732,  0.07106107],
       [ 0.12282863,  0.7215559 , -0.6813738 , -0.63449955],
       [ 0.02935617,  0.6836305 ,  0.7292375 , -0.33270422],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)